In [1]:
from pymongo import MongoClient
import pprint

import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

import requests
from bs4 import BeautifulSoup

import json
import time
import copy

import os


In [3]:
huyler_landing_url = 'https://www.alltrails.com/trail/us/new-jersey/huylers-landing-trail'
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.76 Safari/537.36'} # This is chrome, you can set whatever browser you like
r = requests.get(huyler_landing_url, headers=headers)

In [8]:
client = MongoClient()

db = client.alltrails

In [9]:
pages = db.huyler

In [10]:
pages.insert_one({'link': huyler_landing_url ,'html': r.text})

In [12]:
soup = BeautifulSoup(r.text, 'html.parser')

In [13]:
# Trail title, description, difficulty, info
soup.find_all('h1', class_='xlate-none styles-module__name___1nEtW')
soup.find_all('p', id="auto-overview")
soup.find_all('span', class_="styles-module__diff___22Qtv styles-module__moderate___3w1it styles-module__selected___3fawg")
soup.find_all('span', class_="styles-module__detailData___kQ-eK")

[<span class="styles-module__detailData___kQ-eK">2.2 mi</span>,
 <span class="styles-module__detailData___kQ-eK">410 ft</span>,
 <span class="styles-module__detailData___kQ-eK">Out &amp; back</span>]

In [75]:
written_review = ""
ratings = []
dates = []
types = []
review_text = []
reviewer_links = []

for i, review in enumerate(soup.find_all('div', itemprop="review")):

    ratings.append(soup.find_all('span', class_="MuiRating-root default-module__rating___1k45X MuiRating-sizeLarge MuiRating-readOnly")[i]['aria-label'])
    dates.append(soup.find_all('span', class_="styles-module__dateTrailDetails___3qgZC xlate-none")[i].text.rstrip())
    types.append(soup.find_all('span', class_="styles-module__tag___2s-oD styles-module__activityTag___3-RdN")[i].text.rstrip())
    
    written_review = soup.find_all('div', class_="styles-module__container___3etfA")[i].find('p', itemprop="reviewBody")
    if written_review != None:
        review_text.append(soup.find_all('div', class_="styles-module__container___3etfA")[i].find('p', itemprop="reviewBody").text.rstrip())
        
    reviewer_links.append('alltrails.com' + soup.find_all('div', class_="styles-module__container___3etfA")[1].find('a', class_="styles-module__link___2i6Za styles-module__recording___3jQX3 xlate-none")['href'])


In [ ]:
for review in reviews:
    reviewer_link = 'alltrails.com' + soup.find_all('div', class_="styles-module__container___3etfA")[1].find('a', class_="styles-module__link___2i6Za styles-module__recording___3jQX3 xlate-none")['href']
    soup.find_all('div', class_='data')

In [ ]:
for i,links in enumerate(link):

    # step 1: inspect webpage -- since they have similar structure, I can check out one of the links
    # step 2: get the HTML code via requests library
    sub_page = requests.get(links)
    
    #step 3: put HTML into MongoDB 
    # here I made the key as the link to the page and value the html code
    pages.insert_one({'link':links ,'html': sub_page.text})
    
    # Add time between request to try to prevent this from happening
    print('Website number {}   website link: {}'.format(i,links) )
    time.sleep(2)   # If you request too much code too quickly you can get banned from scrapping the site!
    
    # I have added this so that it does not take to long to run 
    if i > 9:
        break

Post lunch steps:
    —selenium tutorial
    — set scraper for review pages 
        • save pages to mongo
        • get desired info
    — 